In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
def calculate_crc(data):
    crc = 0x0000
    poly = 0x4599  # CRC-15 polynomial

    for bit in data:
        crc ^= (int(bit) & 0x01) << 14  # XOR with the current bit shifted left by 14 bits

        for _ in range(15):
            if crc & 0x8000:
                crc = (crc << 1) ^ poly
            else:
                crc <<= 1

        crc &= 0x7FFF  # Ensure 15 bits

    return crc

In [3]:
def stuff_bits(binary_string):
    result = ''
    count = 0  # Initialize a count for consecutive 0's
    
    for bit in binary_string:
        result += bit  # Append the current bit to the result string
        if bit == '0':
            count += 1  # Increment the count if the current bit is 0
            if count == 5:
                result += '1'  # Insert a 1 after 5 consecutive 0's
                count = 0  # Reset the count after inserting the 1
        else:
            count = 0  # Reset the count if the current bit is not 0
    
    return result

In [4]:
def hex_to_bits(hex_value, num_bits):
    return bin(int(hex_value, 16))[2:].zfill(num_bits)

def convert_to_binary_string(can_id, dlc, data):
    start_of_frame = '0'
    can_id_bits = stuff_bits(hex_to_bits(can_id, 11))
    rtr_bit = '0'
    ide_bit = '0'
    control_r0_bit = '0'
    control_stuff_bit = '1'
    dlc_bits = bin(dlc)[2:].zfill(4)
    data_bits = ''.join(hex_to_bits(hex_byte, 8) for hex_byte in data)
    padding_bits = '0' * (8 * (8 - dlc))  # Fill missing data bytes with zeros
    data_bit_total = stuff_bits(data_bits+padding_bits)
    crc_bit =stuff_bits(bin(calculate_crc(start_of_frame+can_id_bits +rtr_bit+ide_bit+control_r0_bit+control_stuff_bit+ dlc_bits + data_bit_total))[2:].zfill(15))
    crc_delimiter = '1'
    ack_bit = '0'
    ack_delimiter = '1'
    end_of_frame_bits = '1'*7
    inter_Frame_spacing_bits = '1'*3
    return start_of_frame+can_id_bits +rtr_bit+ide_bit+control_r0_bit+ control_stuff_bit+dlc_bits + data_bit_total+crc_bit + crc_delimiter+ack_bit +ack_delimiter+ end_of_frame_bits+inter_Frame_spacing_bits
     


In [5]:
input_filename = "D:\IIT-Delhi\Semester-3\JCD892-M.Tech Major Project Part-1\Implementation\Datasets/vv.txt"
data_array = []
with open(input_filename, 'r') as input_file:
    for line in input_file:
        line=re.sub(r"Timestamp:\s+(\d+\.\d+)\s+ID:\s+([\da-f]{4})\s+\d+\s+DLC:\s+(\d)\s+(\s+[\da-f]{2})*", r"\1 \2 \3 \4", line)
        parts = line.strip().split(' ')
        timestamp = float(parts[0])
        can_id = parts[1]   
        dlc = int(parts[2])
        data = parts[3:3 + dlc]

        converted_data = convert_to_binary_string(can_id, dlc, data)
        data_array.append([timestamp, converted_data])


[1479121434.850202,
 '00110101000000011000000001101001010001000010001100110011011010000010000010000010101000100111111101001101011111111111']

In [16]:
from PIL import Image
import numpy as np
data_rate=512000 #512kbps
# number_of_bits_in_each_frame=114
# frame_duration=number_of_bits_in_each_frame/data_rate
total_bits_per_image=128*128
def create_image(binary_matrix):
    width = len(binary_matrix[0])
    height = len(binary_matrix)
    # pixel_mapping={'3':150,'2':100,'0':0,'1':255}
    color_mapping = {
        '3': (255, 0, 0),  # Red
        '2': (0, 255, 0),  # Blue
        '1': (255, 255, 255),  # White
        '0': (0, 0, 0)  # Black
    }
    image_data = [color_mapping[value] for row in binary_matrix for value in row]
    image = Image.new('RGB', (width, height))
    image.putdata(image_data)
    return image
def calculate_interframe_bits(timestamp_difference, data_rate,i):
    length_of_frame = len(data_array[i])
    frame_duration=length_of_frame/data_rate
    interframe_time = timestamp_difference - frame_duration
    interframe_bits = int(data_rate * interframe_time)
    return '2' * interframe_bits


In [ ]:
binary_matrix = []
a = [['0' for _ in range(128)] for _ in range(128)]
i = 0
x = 0
y = 0

while i < len(data_array):
    bin_str = data_array[i][1]
    for bit in bin_str:
        a[x][y] = bit
        if y == 127:
            x += 1
            y = 0
        else:
            y += 1
        if x == 128:
            binary_matrix.append(a)
            a = [['0' for _ in range(128)] for _ in range(128)]
            x = 0
            y = 0
        
        
        
    if i < len(data_array) - 1:
        timestamp_difference = data_array[i+1][0] - data_array[i][0]
        interframe_bits = calculate_interframe_bits(timestamp_difference, data_rate,i)
        for bit in interframe_bits:
            a[x][y] = bit
            if y == 127:
                x += 1
                y = 0
            else:
                y += 1
            if x == 128:
                binary_matrix.append(a)
                a = [['0' for _ in range(128)] for _ in range(128)]
                x = 0
                y = 0
    while y < 128 and y>0 :
        a[x][y] = '3'
        if y == 127:
            x += 1
            if x == 128:
                binary_matrix.append(a)
                a = [['0' for _ in range(128)] for _ in range(128)]
                x=0
                y=0
            y=0
            break 
        y+=1    

    i += 1

In [7]:
img_matrix=[]
size_of_binary_matrix=len(binary_matrix)
for i in  range (size_of_binary_matrix):
    img_matrix.append(create_image(binary_matrix[i]))

In [8]:
import os
output_folder="D:\IIT-Delhi\Semester-3\JCD892-M.Tech Major Project Part-1\Implementation\Temp_Datasets\Car_Hacking_Dataset\Output_Dataset\images-new-today"
for i,img in enumerate(img_matrix):
    img_path=os.path.join(output_folder,f"image_{i+1}.png")
    img.save(img_path)
print("Images saved")

Images saved


In [9]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv1D,Conv2D,MaxPooling2D
from keras.layers import MaxPool1D
from keras.layers import Flatten
from keras.layers import Dense
from keras.optimizers import Adam
import tensorflow as tf
from PIL import Image
from keras.models import Sequential
from keras.layers import Conv1D, MaxPool1D, Flatten, Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,roc_auc_score
warnings.filterwarnings("ignore")

In [10]:
model=Sequential()
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same",activation="relu",input_shape=(128,128,3)))
model.add(MaxPooling2D(pool_size=(2,2),padding="valid",strides=2))
model.add(Conv2D(filters=256,kernel_size=(3,3),padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),padding="valid",strides=2))
model.add(Dropout(0.9))
model.add(Flatten())
model.add(Dense(units=16,activation="relu"))
model.add(Dense(units=1,activation="sigmoid"))
adam = Adam(learning_rate=0.001)
model.compile(optimizer=adam,loss="binary_crossentropy",metrics=["accuracy"])

In [11]:
image_array_list = [np.array(img) for img in img_matrix]
final_array = np.stack(image_array_list)

In [12]:
final_array.shape

(212, 128, 128, 3)

In [13]:
y = np.array([0]*len(final_array))
y.reshape(1,len(y))

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [14]:
# X_train,X_test,y_train,y_test=train_test_split(final_array,y,test_size=0.2,random_state=True)
# X_train,X_validation,y_train,y_validation=train_test_split(X_train,y_train,test_size=0.2,random_state=True)

In [15]:
X_train = final_array
y_train = y

In [16]:
history = model.fit(X_train, y_train, epochs=3)

Epoch 1/3


7/7 [==============================] - 11s 1s/step - loss: 2.8927 - accuracy: 0.9387
Epoch 2/3
7/7 [==============================] - 11s 1s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/3
7/7 [==============================] - 9s 1s/step - loss: 0.0000e+00 - accuracy: 1.0000


In [39]:
import numpy as np
import pandas as pd
import re

In [40]:
input_filename = "D:\IIT-Delhi\Semester-3\JCD892-M.Tech Major Project Part-1\Implementation\Temp_Datasets\Car_Hacking_Dataset\Input_Dataset\DoS_datset_insertions.csv"
data_array1 = []
frame_type = []
with open(input_filename, 'r') as input_file:
    for line in input_file:
        # line=re.sub(r"Timestamp:\s+(\d+\.\d+)\s+ID:\s+([\da-f]{4})\s+\d+\s+DLC:\s+(\d)\s+(\s+[\da-f]{2})*", r"\1 \2 \3 \4", line)
        parts = line.strip().split(',')
        timestamp = float(parts[0])
        can_id = parts[1]   
        dlc = int(parts[2])
        data = parts[3:3 + dlc]
        frame_type.append(0 if parts[-1]=='R' else 1)
        converted_data = convert_to_binary_string(can_id, dlc, data)
        data_array1.append([timestamp, converted_data])



In [41]:
from PIL import Image
import numpy as np
data_rate=512000 #512kbps
# number_of_bits_in_each_frame=114
# frame_duration=number_of_bits_in_each_frame/data_rate
total_bits_per_image=128*128
def create_image_new(binary_matrix):
    width = len(binary_matrix[0])
    height = len(binary_matrix)
    # pixel_mapping={'3':150,'2':100,'0':0,'1':255}
    color_mapping = {
        '3': (255, 0, 0),  # Red
        '2': (0, 255, 0),  # Blue
        '1': (255, 255, 255),  # White
        '0': (0, 0, 0)  # Black
    }
    image_data = [color_mapping[value] for row in binary_matrix for value in row]
    image = Image.new('RGB', (width, height))
    image.putdata(image_data)
    return image
def calculate_interframe_bits_new(timestamp_difference, data_rate,i):
    length_of_frame = len(data_array1[i])
    frame_duration=length_of_frame/data_rate
    interframe_time = timestamp_difference - frame_duration
    interframe_bits = int(data_rate * interframe_time)
    return '2' * interframe_bits


In [42]:

binary_matrix1 = []
y1=[]
a = [['0' for _ in range(128)] for _ in range(128)]
i = 0
x = 0
y = 0
flag = 0
while i < len(data_array1):
    bin_str = data_array1[i][1]
    if frame_type[i]==1:
        flag =  1
    for bit in bin_str:
        a[x][y] = bit
        if y == 127:
            x += 1
            y = 0
        else:
            y += 1
        if x == 128:
            binary_matrix1.append(a)
            y1.append(1 if flag == 1 else 0)
            falg = 0
            a = [['0' for _ in range(128)] for _ in range(128)]
            x = 0
            y = 0
        
        
        
    if i < len(data_array1) - 1:
        timestamp_difference = data_array1[i+1][0] - data_array1[i][0]
        interframe_bits = calculate_interframe_bits_new(timestamp_difference, data_rate,i)
        for bit in interframe_bits:
            a[x][y] = bit
            if y == 127:
                x += 1
                y = 0
            else:
                y += 1
            if x == 128:
                binary_matrix1.append(a)
                y1.append(1 if flag == 1 else 0)
                flag = 0
                a = [['0' for _ in range(128)] for _ in range(128)]
                x = 0
                y = 0
    while y < 128 and y>0 :
        a[x][y] = '3'
        if y == 127:
            x += 1
            if x == 128:
                binary_matrix1.append(a)
                y1.append(1 if flag == 1 else 0)
                falg = 0
                a = [['0' for _ in range(128)] for _ in range(128)]
                x=0
                y=0
            y=0
            break 
        y+=1    

    i += 1




286

In [43]:
img_matrix1=[]
size_of_binary_matrix1=len(binary_matrix1)
for i in  range (size_of_binary_matrix1):
    img_matrix1.append(create_image(binary_matrix1[i]))
    # output_folder="D:\IIT-Delhi\Semester-3\JCD892-M.Tech Major Project Part-1\Implementation\Temp_Datasets\Car_Hacking_Dataset\Output_Dataset/test"
    # for i,img in enumerate(img_matrix1):
    # img_path=os.path.join(output_folder,f"image_{i+1}.png")
    # img.save(img_path)
print("Images saved")
image_array_list1 = [np.array(img) for img in img_matrix1]
final_array1 = np.stack(image_array_list1)

Images saved


In [44]:
y_label = np.array(y1)
len(y_label)

6749

In [45]:
np.count_nonzero(y1)

3868

In [46]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv1D,Conv2D,MaxPooling2D
from keras.layers import MaxPool1D
from keras.layers import Flatten
from keras.layers import Dense
from keras.optimizers import Adam
import tensorflow as tf
from PIL import Image
from keras.models import Sequential
from keras.layers import Conv1D, MaxPool1D, Flatten, Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,roc_auc_score
warnings.filterwarnings("ignore")

In [47]:
model=Sequential()
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same",activation="relu",input_shape=(128,128,3)))
model.add(MaxPooling2D(pool_size=(2,2),padding="valid",strides=2))
model.add(Conv2D(filters=256,kernel_size=(3,3),padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),padding="valid",strides=2))
model.add(Dropout(0.9))
model.add(Flatten())
model.add(Dense(units=16,activation="relu"))
model.add(Dense(units=1,activation="sigmoid"))
adam = Adam(learning_rate=0.001)
model.compile(optimizer=adam,loss="binary_crossentropy",metrics=["accuracy"])

In [48]:
y_label.reshape(1,len(y_label))
print(y_label.shape)


(6749,)


In [49]:
X_train,X_test,y_train,y_test=train_test_split(final_array1,y_label,test_size=0.2)

In [50]:
history = model.fit(X_train, y_train, epochs=15)
loss, accuracy = model.evaluate(X_test, y_test)

Epoch 1/15
169/169 [==============================] - 291s 2s/step - loss: 2.4849 - accuracy: 0.5631
Epoch 2/15
169/169 [==============================] - 315s 2s/step - loss: 0.6859 - accuracy: 0.5766
Epoch 3/15
169/169 [==============================] - 320s 2s/step - loss: 0.6849 - accuracy: 0.5764
Epoch 4/15
169/169 [==============================] - 313s 2s/step - loss: 0.6824 - accuracy: 0.5766
Epoch 5/15
169/169 [==============================] - 312s 2s/step - loss: 0.6818 - accuracy: 0.5766
Epoch 6/15
169/169 [==============================] - 307s 2s/step - loss: 0.6816 - accuracy: 0.5766
Epoch 7/15
129/169 [=====================>........] - ETA: 1:11 - loss: 0.6824 - accuracy: 0.5734

KeyboardInterrupt: 

In [1]:
loss, accuracy = model.evaluate(X_test, y_test)

NameError: name 'model' is not defined

2/2 [==============================] - 1s 397ms/step


In [38]:
from sklearn.metrics import classification_report,roc_auc_score,auc,roc_curve
y_score=model.predict(X_test)
y_pred=np.round(y_score)
target_names = ["Normal","Attack"]
print(classification_report(y_test, y_pred,target_names=target_names))

2/2 [==============================] - 1s 381ms/step
              precision    recall  f1-score   support

      Normal       0.00      0.00      0.00        13
      Attack       0.78      1.00      0.87        45

    accuracy                           0.78        58
   macro avg       0.39      0.50      0.44        58
weighted avg       0.60      0.78      0.68        58



In [32]:
print(y_pred)

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
